In [1]:
import json
import pydgraph

from pydgraph import DgraphClient, DgraphClientStub
from grapl_analyzerlib.schemas import *

from grapl_analyzerlib.schemas.schema_builder import ManyToMany

In [2]:

def set_schema(client, schema):
    op = pydgraph.Operation(schema=schema)
    client.alter(op)
                 
                
def drop_all(client):
    op = pydgraph.Operation(drop_all=True)
    client.alter(op)
    
def format_schemas(schema_defs):
    schemas = "\n\n".join([schema.to_schema_str() for schema in schema_defs])

    types = "\n\n".join([schema.generate_type() for schema in schema_defs])

    return "\n".join([
        "  # Type Definitions",
        types,
        "\n  # Schema Definitions",
        schemas,
    ])

In [3]:
mclient = DgraphClient(DgraphClientStub('master_graph:9080'))
eclient = DgraphClient(DgraphClientStub('engagement_graph:9080'))

In [8]:
schemas = (
    AssetSchema(),
    ProcessSchema(),
    FileSchema(),
    IpConnectionSchema(),
    IpAddressSchema(),
    IpPortSchema(),
    NetworkConnectionSchema(),
    ProcessInboundConnectionSchema(),
    ProcessOutboundConnectionSchema(),  
)

schema_str = format_schemas(schemas)
set_schema(mclient, schema_str)


risk_schema = RiskSchema()
lens_schema = LensSchema()
eg_schemas = [s.with_forward_edge('risks', ManyToMany(RiskSchema), 'risky_nodes') for s in schemas]
eg_schemas.extend([risk_schema, lens_schema])
eg_schema_str = format_schemas(eg_schemas)
set_schema(eclient, eg_schema_str)


  # Type Definitions
  type Asset {
      node_key: string
      hostname: string
      risks: uid  # type: Risk
  }


  type Process {
      node_key: string
      asset_id: string
      image_name: string
      process_name: string
      arguments: string
      process_id: int
      created_timestamp: int
      terminate_time: int
      children: uid  # type: Process
      bin_file: uid  # type: File
      created_files: uid  # type: File
      deleted_files: uid  # type: File
      read_files: uid  # type: File
      wrote_files: uid  # type: File
      created_connections: uid  # type: ProcessOutboundConnection
      inbound_connections: uid  # type: ProcessInboundConnection
      risks: uid  # type: Risk
  }


  type File {
      node_key: string
      file_name: string
      asset_id: string
      file_path: string
      file_extension: string
      file_mime_type: string
      file_version: string
      file_description: string
      file_product: string
      file_company: stri